# Parallel Programming with Python

Python has the ability to run in parallel, using both shared memory and distributed memory methods.  This tutorial is meant to give you a brief introduction to what's available and, more importantly, when it's appropriate to use.

## Python Threading

Generally, Python threading is terrible.  But it shouldn't be:

* POSIX threads
* Shared memory with parent process
* Lightweight threads

### Global Interpreter Lock (GIL)

In order to keep memory coherent, the Python intrepter only allows a single thread to run at once....killing performance for any kind of shared memory workload.

There are (some) good reasons for this (I/O, intrepreter maintenance, etc.)

### Example: Calculate Pi with Python Threads

Simple process:
* Inscribe a circle in a square
* Throw darts at it
* Count how many are inside the circle and how many are outside
* Use the ratio of those to compute pi

<img src="../img/circle_and_square.png" style="height:350px">

In [1]:
from threading import Thread, Lock
import random

lock = Lock() # lock for making operations atomic

def calcInside(nsamples,rank):
    global inside # we need something everyone can share random.seed(rank)
    random.seed(rank)
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x*x)+(y*y)<1:
            lock.acquire() # GIL doesn't always save you
            inside += 1
            lock.release()

if __name__ == '__main__':
    nt=4 # thread count
    inside = 0 # initialise
    samples=int(12e6/nt)
    threads=[Thread(target=calcInside, args=(samples,i)) for i in range(nt)]
    
    for t in threads: t.start()
    for t in threads: t.join()
    
    print((4.0*inside)/(1.0*samples*nt))

3.141045666666667


## Subprocess

Python's `subprocess` module allows the Python intrepter to spawn and control processes that aren't affected by the GIL.  The basic command in the `subprocess` module is `Popen()`, which lets you open a proces:

In [2]:
import subprocess
pi=subprocess.Popen('python -c "import math; print(math.pi)"',shell=True,stdout=subprocess.PIPE)

In [3]:
pi.stdout.read()

b'3.141592653589793\n'

In [15]:
pi.pid

52124

Some issues with subprocess:
* Shared memory is tricky at best
* Locks and atomics are difficult

There are better ways of doing this...

## Multiprocessing

This module blends together Python threads and subprocesses.  It bypasses the GIL, so threads can be used and see some performance.  Under the hood it uses subprocesses, but has a manager to handle things like synchronization and distribuited sharing (but still not true shared memory).

### Calculating pi with Multiprocessing

We will use the `Pool` module to calculate pi.  `Pool` allows you to define a group of worker processes that you will then divide some work amongst.  `Pool` takes two inputs:

* A function that we want to run across the pool of workers
* An iterable...some way to identify how we're splitting up work

In [4]:
import multiprocessing as mp
import numpy as np
import random

processes = mp.cpu_count()
nsamples = int(12e6/processes)

def calcInside(rank):
    inside = 0
    random.seed(rank)
    for i in range(nsamples):
        x = random.random();
        y = random.random();
        if (x*x)+(y*y)<1:
            inside += 1
    return (4.0*inside)/nsamples

# Important to check if main so child processes don't try to run it
if __name__ == '__main__':
    pool = mp.Pool(processes)
    result = pool.map(calcInside, range(processes))
    print(np.mean(result))

3.141337666666667


The `Multiprocessing` module has support for other parallel constructs like process communication and locks.  We won't go into them today, but you should be aware of them.

While Multiprocessing is certainly an improvement over `subprocess` and Python threads, it does come with overhead that impacts performance.  Additionally, it will only scale to a single node (no distributed memory capability).

In order to do that, we need...

## mpi4py

`mpi4py` is a set of bindings to make use of MPI, Message Passing Interface.  MPI forms the basis of most applications that run on HPC systems today.  We won't cover MPI today, but it is important to understand a few basics to understand what `mpi4py` is doing.

Simply, all MPI allows is for processors to communicate data between each other.   Each process executes the same instructions (or code), but on different parts of the data.  At points throughout the computation, they may need to send or receive data to/from memory locations that are non-local.  MPI is the API that allows for this.

### Hello World

In [5]:
from mpi4py import MPI
import sys

size = MPI.COMM_WORLD.Get_size()
rank = MPI.COMM_WORLD.Get_rank()
name = MPI.Get_processor_name()

sys.stdout.write(
    "Hello, World! I am process %d of %d on %s.\n"
    % (rank, size, name))

Hello, World! I am process 0 of 1 on turing.


In [6]:
!mpirun -np 4 python ../code/mpi4py/helloworld.py

Hello, World! I am process 1 of 4 on turing.
Hello, World! I am process 0 of 4 on turing.
Hello, World! I am process 2 of 4 on turing.
Hello, World! I am process 3 of 4 on turing.


### Point-to-Point Communication

In [7]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
elif rank == 1:
    data = comm.recv(source=0)
    print('On process 1, data is ',data)

Exception: Invalid rank, error stack:
MPI_Send(174): MPI_Send(buf=0x7fb96a9272a0, count=35, MPI_BYTE, dest=1, tag=0, MPI_COMM_WORLD) failed
MPI_Send(100): Invalid rank has value 1 but must be nonnegative and less than 1

In [8]:
!mpirun -np 4 python ../code/mpi4py/pt2pt.py

On process 1, data is  {'a': 7, 'b': 3.14}


We sent a dictionary, but we can also send NumPy arrays (and we should try to do that all the time):

In [28]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # in real code, this section might
    # read in data parameters from a file
    numData = 10  
    comm.send(numData, dest=1)

    data = np.linspace(0.0,3.14,numData)  
    comm.Send(data, dest=1)

elif rank == 1:

    numData = comm.recv(source=0)
    print('Number of data to receive: ',numData)

    data = np.empty(numData, dtype='d')  # allocate space to receive the array
    comm.Recv(data, source=0)

    print('data received: ',data)

Exception: Invalid rank, error stack:
MPI_Send(174): MPI_Send(buf=0x7ff23076a5c0, count=5, MPI_BYTE, dest=1, tag=0, MPI_COMM_WORLD) failed
MPI_Send(100): Invalid rank has value 1 but must be nonnegative and less than 1

In [30]:
!mpirun -np 4 python ../code/mpi4py/pt2pt_numpy.py

Number of data to receive:  10
data received:  [0.         0.34888889 0.69777778 1.04666667 1.39555556 1.74444444
 2.09333333 2.44222222 2.79111111 3.14      ]


### Collectives

Collectives are operations that all processors execute together.  They may execute at slightly different times, but they all will call the same function.  These are useful for operations like gathering data onto a root process, or distributing data from one to all.

Hers' an example of performing a `gather` operation:

<img src="../img/gather.png" style="height:150px">

In [32]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()   

numDataPerRank = 10  
sendbuf = np.linspace(rank*numDataPerRank+1,(rank+1)*numDataPerRank,numDataPerRank)
print('Rank: ',rank, ', sendbuf: ',sendbuf)

recvbuf = None
if rank == 0:
    recvbuf = np.empty(numDataPerRank*size, dtype='d')  

comm.Gather(sendbuf, recvbuf, root=0)

if rank == 0:
    print('Rank: ',rank, ', recvbuf received: ',recvbuf)

Rank:  0 , sendbuf:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Rank:  0 , recvbuf received:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [33]:
!mpirun -np 4 python ../code/mpi4py/gather.py

Rank:  3 , sendbuf:  [31. 32. 33. 34. 35. 36. 37. 38. 39. 40.]
Rank:  2 , sendbuf:  [21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
Rank:  1 , sendbuf:  [11. 12. 13. 14. 15. 16. 17. 18. 19. 20.]
Rank:  0 , sendbuf:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Rank:  0 , recvbuf received:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40.]


mpi4py has the ability to ship *any* serialisable Python object.  That means that objects like `dicts` need to be converted to a byte stream, a process called pickling.  That means a Python object (except for strings and ints) needs to be pickled, sent over MPI, and then repickled...adding significant overhead.

However, arrays in NumPy map to C memory allocations, and mpi4py can send them at *almost* the speed of C/C++/Fortran.

## Python at Pawsey

Pawsey has a number of solutions for Python users:

- Compiled Python modules (Versions 2&3)
- Tuned NumPy/SciPy libraries (linked agains MKL and Cray-LibSci)
- Job-Packing Methods
- Shifter/Singularity

### Job-Packing

Users of Magnus and Galaxy are allocated an entire node, and charged accordingly, whether they use it all or not.  Many users want to run as many single-core Python jobs on a node as possible.  The easiest way to do that is to use job-packing in your SLURM jobscript.

```console
#!/bin/bash -l
#SBATCH --nodes=1
#SBATCH --ntasks=24
#SBATCH --ntasks-per-node=24
#SBATCH --cpus-per-task=1
#SBATCH --time=00:10:00
#SBATCH --partition=debugq
#SBATCH --account=pawsey0001
#SBATCH --export=NONE

module swap PrgEnv-crady PrgEnv-gnu
module load python
module load numpy
module load scipy
module load matplotlib

srun --export=ALL -n 24 -N 1 python_job_wrapper.sh
```

We run a single wrapper script across 24 cores.  The key is how we write our wrapper script:

```
#!/bin/bash

python voxelSlice.py qs-curie-${SLURM_PROCID}"
```

Each instance of the wrapper script will call the Python interpreter, but we use the environment variable `SLURM_PROCID` to differentiate between the cores, and each core takes a different input data set.

The benefit with this method is it usually require no changes to existing Python scripts, but may require some thought be given as to how to structure data inputs.

### Containers

Pawsey also has Docker images available to use, particularly for Python users.  We have a program called Shifter installed on our Cray systems.  It allows for Docker containers to be run on a shared HPC system, while still maintaining performance.

<img src="../img/Shifter_OSU_allgather.png" style="height:450px">

<img src="../img/Shifter_OSU_bandwidth_reduced.png" style="height:450px">

Job scripts require minimal modification:
    
```
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH --image=docker:pawsey/hpc-python:latest
 
 
module load shifter
 
 
srun -n 24 shifter python my_python_app.py <args>
```

And the Docker images provide a base of what most users would need to build their own images:
    
```
FROM ubuntu:latest

LABEL maintainer="brian.skjerven@pawsey.org.au"

RUN apt-get update \
      && apt-get install -y \
      cython \
      python-minimal \
      python-pip

RUN pip install --upgrade pip \
      && pip install \
      astropy \
      h5py \
      matplotlib \
      nose \
      numpy \
      pytest \
      scipy \
      setuptools

CMD ["/bin/bash"]
```

The other benefit to using Python in a container is related to dynamic library loading:

<img src="../img/shifter_magnus.png" style="height:450px">

## Final Thoughts

- Make use of Pawsey compiled Python libraries (performance and module compatibility)
- Try to use MPI capable libraries
- Multiprocess *can* be useful, but there is a performance hit
- Other Python options available to users (Shifter, job-packing)